In [ ]:
# Autoreload modules
%load_ext autoreload
%autoreload 2

In [ ]:
# To have access to moduels
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('')))

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import Accuracy

from torchvision.transforms import Compose, GaussianBlur, RandomRotation, RandomChoice, RandomApply, RandomAffine
from dataloader.transforms import GaussianNoise

from copy import deepcopy

from colorama import Fore

from matplotlib import pyplot as plt

from dataloader.dataset import ADNI
from dataloader.dataloader import ADNILoader

from utils.utils import count_parameters, save_model
from utils.report import cnn_report

# Dataset and Dataloader Setup

In [ ]:
image_size = (95, 79)

gaussian_blur = GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2))
gaussian_noise = GaussianNoise(mean=0, std=0.1)
random_rotation = RandomRotation(degrees=5)
random_translate = RandomAffine(degrees=0, translate=(0.05, 0.05))
random_choice = RandomChoice([gaussian_blur, gaussian_noise, random_rotation, random_translate])
random_transforms = RandomApply([random_choice], p=0.5)

train_transforms = Compose([random_transforms])
valid_transforms = Compose([])
test_transforms = Compose([])

In [ ]:
train_ds = ADNI("../Data/Training/", transforms=train_transforms, extra_channel_dim=True)
valid_ds = ADNI("../Data/Validation/", transforms=valid_transforms, extra_channel_dim=True)
test_ds = ADNI("../Data/Test/", transforms=test_transforms, extra_channel_dim=True)

In [ ]:
idx = 0
image, label = train_ds[idx]

print("Image shape:", image.shape)
print("Label:", label.item())

print("Number of training samples:", len(train_ds))
print("Number of validation samples:", len(valid_ds))
print("Number of test samples:", len(test_ds), "\n")

image = image[0, :, :]
fig, axes = plt.subplots(nrows=6, ncols=10, figsize=(3, 2), dpi=300)
for row in range(6):
    for col in range(10):
        idx = row * 10 + col
        axes[row, col].imshow(image[idx, :, :])
        axes[row, col].axis("off");
        # print(image[idx, :, :].min(), image[idx, :, :].max())

In [ ]:
id2label = {0: "CN", 1: "MCI", 2: "AD"}
label2id = {"CN": 0, "MCI": 1, "AD": 2}

print(id2label[label.item()])

In [ ]:
train_batch_size = 15
valid_batch_size = 15
test_batch_size = 15

hparams = {'train_ds': train_ds,
           'valid_ds': valid_ds,
           'test_ds': test_ds,
           'train_batch_size': train_batch_size,
           'valid_batch_size': valid_batch_size,
           'test_batch_size': test_batch_size,
           'num_workers': 20
          }

train_dataloader = ADNILoader(**hparams).train_dataloader()
valid_dataloader= ADNILoader(**hparams).validation_dataloader()
test_dataloader = ADNILoader(**hparams).test_dataloader()

batch = next(iter(train_dataloader))
print(batch[0].shape)
print(batch[1].shape)

# Model Development

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_labels=3):
        super(CNN, self).__init__()
        self.num_labels = num_labels
        
        self.model = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=5, kernel_size=(3, 3, 3)),
            nn.ReLU(),            
            nn.Conv3d(in_channels=5, out_channels=10, kernel_size=(3, 3, 3)),
            nn.ReLU(),
            nn.Dropout3d(0.15),
            nn.MaxPool3d(kernel_size=(3, 3, 3)),
            nn.BatchNorm3d(num_features=10),
            
            nn.Conv3d(in_channels=10, out_channels=15, kernel_size=(3, 3, 3)),
            nn.ReLU(),            
            nn.Conv3d(in_channels=15, out_channels=20, kernel_size=(3, 3, 3)),
            nn.ReLU(),
            nn.Dropout3d(0.15),
            nn.MaxPool3d(kernel_size=(3, 3, 3)),
            nn.BatchNorm3d(num_features=20),
            
            nn.Flatten(),
            
            nn.Linear(4480, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 3),
        )
        
    def forward(self, x):
        return self.model(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

model = CNN(3).to(device)

optimizer = Adam(model.parameters(), lr=1e-4, weight_decay=0.15)

class_0_freq = 140
class_1_freq = 160
class_2_freq = 160
weight = torch.tensor([1/class_0_freq, 1/class_1_freq, 1/class_2_freq]).to(device)
criterion = nn.CrossEntropyLoss(weight)

accuracy = Accuracy()
writer = SummaryWriter()
scheduler = ExponentialLR(optimizer, gamma=0.99)

In [ ]:
epochs = 200
train_accs = []
valid_accs = []
train_losses = []
valid_losses = []
best_loss = 100
best_acc = 0
saved = False

for epoch in range(epochs):
    print(Fore.YELLOW + f"Epoch: {(epoch+1):02}/{epochs}")
    for step, (x, y) in enumerate(train_dataloader):
        x, y  = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        preds = logits.argmax(1)
        acc = accuracy(y.cpu(), preds.cpu())
        optimizer.zero_grad()           
        loss.backward()                 
        optimizer.step()
        train_losses.append(loss.item())
        train_accs.append(acc.item())
    
        if (step % 12 == 0) or (step == len(train_dataloader)):
            train_loss = sum(train_losses)/len(train_losses)
            train_acc = sum(train_accs)/len(train_accs)
            writer.add_scalar('train_loss', train_loss, epoch * len(train_dataloader) + step)
            writer.add_scalar('train_acc', train_acc, epoch * len(train_dataloader) + step)
            train_losses.clear()
            train_accs.clear()
            
            model.eval() 
            with torch.no_grad():
                for x, y in valid_dataloader:
                    x, y  = x.to(device), y.to(device)
                    logits = model(x)
                    loss = criterion(logits, y)
                    preds = logits.argmax(1)
                    acc = accuracy(y.cpu(), preds.cpu())
                    valid_losses.append(loss.item())
                    valid_accs.append(acc.item())
            
            valid_loss = sum(valid_losses)/len(valid_losses)
            valid_acc = sum(valid_accs)/len(valid_accs)
            writer.add_scalar('valid_loss', valid_loss, epoch * len(train_dataloader) + step)
            writer.add_scalar('valid_acc', valid_acc, epoch * len(train_dataloader) + step)
            valid_losses.clear()
            valid_accs.clear()
            
            if best_loss > valid_loss:
                best_loss = valid_loss
                best_model_loss = deepcopy(model.state_dict())
                saved = True
                
            if best_acc < valid_acc:
                best_acc = valid_acc
                best_model_acc = deepcopy(model.state_dict())
                saved = True
                
            if saved:
                print(Fore.GREEN + f"Training Loss(Accuracy): {train_loss:.2f}({train_acc:.2f}), Validation Loss(Accuracy): {valid_loss:.2f}({valid_acc:.2f})")
                saved = False
            else:
                print(Fore.RED + f"Training Loss(Accuracy): {train_loss:.2f}({train_acc:.2f}), Validation Loss(Accuracy): {valid_loss:.2f}({valid_acc:.2f})")

            model.train()
    
    scheduler.step()
    
    print(Fore.YELLOW + "=" * 74)

# Model Save and Load

In [ ]:
save_model(best_model_acc, "Best models/", "best_model_3D.pt")

In [ ]:
model.load_state_dict(torch.load("Best models/best_model_3D.pt"))

# Evaluation

In [ ]:
cnn_report(model, valid_dataloader, device)

In [ ]:
cnn_report(model, test_dataloader, device)